In [27]:
%pip install matplotlib
%pip install seaborn
%pip install numpy
%pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
    ERROR: Command errored out with exit status 1:
     command: /Library/Developer/CommandLineTools

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

/var/folders/gd/0dt483qd2cg3cqgcsrx3pnkr0000gp/T/ipykernel_31634/3381811058.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [5]:
# function that will combine train and test set, creating a column 'is_train'
# for later when we separate them back
def concat_df(train, test):
    test['is_train'] = False
    train['is_train'] = True
    combined = pd.concat([train, test], sort=False)
    return combined

# function that will separate the combined df
def separate_df(combined):
    train = combined[combined['is_train'] == True].drop('is_train', axis=1)
    test = combined[combined['is_train'] == False].drop('is_train', axis=1)
    return train, test

In [6]:
# combine test and train data to preprocess the data all together.
df_combined = concat_df(df_train, df_test)

In [7]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12970 entries, 0 to 4276
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  object 
 2   CryoSleep     12660 non-null  object 
 3   Cabin         12671 non-null  object 
 4   Destination   12696 non-null  object 
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  object 
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Name          12676 non-null  object 
 13  Transported   8693 non-null   object 
 14  is_train      12970 non-null  bool   
dtypes: bool(1), float64(6), object(8)
memory usage: 1.5+ MB


In [8]:
# If a person is in cryosleep, they would not be able to spend any money so
# we can fill null with 0 for money spent columns and false for VIP column
money_col = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
vip = ['VIP']
condition = (df_combined['CryoSleep'] == True)

df_combined.loc[condition, money_col] = df_combined.loc[condition, money_col].fillna(0)
df_combined.loc[condition, vip] = df_combined.loc[condition, vip].fillna(False)

/var/folders/gd/0dt483qd2cg3cqgcsrx3pnkr0000gp/T/ipykernel_31634/1822519467.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined.loc[condition, vip] = df_combined.loc[condition, vip].fillna(False)


In [10]:
# If a person have not spent any money, it is likely that person is in cryosleep.
cryo = ['CryoSleep']

# the condition is True if all the money_col values are 0
condition = (df_combined[money_col].eq(0).all(axis=1))

df_combined.loc[condition, cryo] = df_combined.loc[condition, cryo].fillna(True)
df_combined['CryoSleep'].fillna(False, inplace=True)

/var/folders/gd/0dt483qd2cg3cqgcsrx3pnkr0000gp/T/ipykernel_31634/3845885266.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined.loc[condition, cryo] = df_combined.loc[condition, cryo].fillna(True)
/var/folders/gd/0dt483qd2cg3cqgcsrx3pnkr0000gp/T/ipykernel_31634/3845885266.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on 

In [11]:
df_combined.isna().sum()

PassengerId        0
HomePlanet       288
CryoSleep          0
Cabin            299
Destination      274
Age              270
VIP              192
RoomService      170
FoodCourt        180
ShoppingMall     175
Spa              177
VRDeck           177
Name             294
Transported     4277
is_train           0
dtype: int64

In [12]:
# since 'Cabin' column has three components, we can slit them into three columns

cabin_split = df_combined['Cabin'].str.split('/', expand=True)
cabin_split.columns = ['Deck', 'RoomNumber', 'Side']
ndf = pd.concat([df_combined, cabin_split], axis=1)
df_combined = ndf
df_combined.drop(columns=['Cabin'], inplace=True)


In [14]:
group_means = df_combined.groupby(['HomePlanet', 'CryoSleep', 'Destination'])['Age'].mean()
print(group_means)

HomePlanet  CryoSleep  Destination  
Earth       False      55 Cancri e      24.101341
                       PSO J318.5-22    29.400391
                       TRAPPIST-1e      27.525610
            True       55 Cancri e      21.093960
                       PSO J318.5-22    24.065913
                       TRAPPIST-1e      23.274959
Europa      False      55 Cancri e      36.318328
                       PSO J318.5-22    36.307692
                       TRAPPIST-1e      35.037146
            True       55 Cancri e      32.905775
                       PSO J318.5-22    33.562500
                       TRAPPIST-1e      32.797950
Mars        False      55 Cancri e      26.369048
                       PSO J318.5-22    35.140000
                       TRAPPIST-1e      30.036870
            True       55 Cancri e      27.561983
                       PSO J318.5-22    40.533333
                       TRAPPIST-1e      28.753172
Name: Age, dtype: float64


In [17]:
# fill in NA values for Age column

group_means = df_combined.groupby(['CryoSleep', 'HomePlanet', 'Destination'], group_keys=True)['Age'].mean().reset_index(name='Age_mean')
df_combined = df_combined.merge(group_means, on=['CryoSleep', 'HomePlanet', 'Destination'], how='left')
df_combined['Age'] = df_combined['Age'].fillna(df_combined['Age_mean'])
df_combined.drop(columns=['Age_mean'], inplace=True)

In [19]:
pd.pivot_table(df_combined, index='Transported', columns='Deck', values='PassengerId', aggfunc='count')

Deck,A,B,C,D,E,F,G,T
Transported,,,,,,,,
False,129,207,239,271,563,1565,1238,4
True,127,572,508,207,313,1229,1321,1


In [20]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  object 
 2   CryoSleep     12970 non-null  bool   
 3   Destination   12696 non-null  object 
 4   Age           12957 non-null  float64
 5   VIP           12778 non-null  object 
 6   RoomService   12800 non-null  float64
 7   FoodCourt     12790 non-null  float64
 8   ShoppingMall  12795 non-null  float64
 9   Spa           12793 non-null  float64
 10  VRDeck        12793 non-null  float64
 11  Name          12676 non-null  object 
 12  Transported   8693 non-null   object 
 13  is_train      12970 non-null  bool   
 14  Deck          12671 non-null  object 
 15  RoomNumber    12671 non-null  object 
 16  Side          12671 non-null  object 
dtypes: bool(2), float64(6), object(9)
memory usage: 1.5+ MB


In [25]:
# since the data for money_col is very skewed, take log and then standardize
# taking log

for column in money_col:
    df_combined[column + '_log'] = np.log1p(df_combined[column])


In [28]:
from sklearn.preprocessing import StandardScaler

In [29]:
# standardization
scaler = StandardScaler()

money_col_log = ['RoomService_log', 'FoodCourt_log', 'ShoppingMall_log', 'Spa_log', 'VRDeck_log']
scaled_columns = scaler.fit_transform(df_combined[money_col_log])
for i, col in enumerate(money_col_log):
    df_combined[col + '_scaled'] = scaled_columns[:, i]

In [31]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   PassengerId              12970 non-null  object 
 1   HomePlanet               12682 non-null  object 
 2   CryoSleep                12970 non-null  bool   
 3   Destination              12696 non-null  object 
 4   Age                      12957 non-null  float64
 5   VIP                      12778 non-null  object 
 6   RoomService              12800 non-null  float64
 7   FoodCourt                12790 non-null  float64
 8   ShoppingMall             12795 non-null  float64
 9   Spa                      12793 non-null  float64
 10  VRDeck                   12793 non-null  float64
 11  Name                     12676 non-null  object 
 12  Transported              8693 non-null   object 
 13  is_train                 12970 non-null  bool   
 14  Deck                  

In [35]:
# change boolean variable cryosleep to integer values
df_combined['CryoSleep'] = df_combined['CryoSleep'].astype(int)

In [36]:
# perform one-hot encoding for categorical variables that will be used

df_combined = pd.get_dummies(df_combined, columns=['HomePlanet', 'Deck', 'Side', 'Destination'])

In [37]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                12970 non-null  object 
 1   CryoSleep                  12970 non-null  int64  
 2   Age                        12957 non-null  float64
 3   VIP                        12778 non-null  object 
 4   RoomService                12800 non-null  float64
 5   FoodCourt                  12790 non-null  float64
 6   ShoppingMall               12795 non-null  float64
 7   Spa                        12793 non-null  float64
 8   VRDeck                     12793 non-null  float64
 9   Name                       12676 non-null  object 
 10  Transported                8693 non-null   object 
 11  is_train                   12970 non-null  bool   
 12  RoomNumber                 12671 non-null  object 
 13  RoomService_log            12800 non-null  flo